# 📈 Modelo de Optimización con Gurobi
Este notebook resuelve el problema de asignación de horas a proyectos usando programación lineal con Gurobi.

In [1]:
from gurobipy import Model, GRB, quicksum

In [6]:
# === Datos de entrada ===
trabajadores = ['Juan', 'Pedro', 'María', 'Luciana', 'Claudia', 'Javier', 'Ignacia', 'Lucas', 'Mariana', 'Mateo']
proyectos = ['Rangekuta', 'Tripamonti', 'Omega', 'Villa Zeckers', 'Lo Redal', 'Alto Trufen']

h = {'Juan': 10, 'Pedro': 10, 'María': 7, 'Luciana': 15, 'Claudia': 14, 'Javier': 20, 'Ignacia': 9,'Lucas': 15, 'Mariana': 10, 'Mateo': 6 }
l = {'Rangekuta': 6, 'Tripamonti': 10, 'Omega': 7, 'Villa Zeckers': 11, 'Lo Redal': 9, 'Alto Trufen':  13}
u = {'Rangekuta': 17, 'Tripamonti': 20, 'Omega': 22, 'Villa Zeckers': 22, 'Lo Redal': 20, 'Alto Trufen': 28}
b = {'Rangekuta': 450, 'Tripamonti': 520, 'Omega': 310, 'Villa Zeckers': 440, 'Lo Redal': 468, 'Alto Trufen': 1170}
a = {'Rangekuta': 80, 'Tripamonti': 130, 'Omega': 70, 'Villa Zeckers': 140, 'Lo Redal': 110, 'Alto Trufen': 100}

# Costo fijo para todos por ahora
c = {(i, j): 30 for i in trabajadores for j in proyectos}

In [7]:
# === Modelo ===
model = Model("Asignación de Horas")
x = model.addVars(trabajadores, proyectos, name="x", lb=0)

# Restricciones
for i in trabajadores:
    model.addConstr(quicksum(x[i, j] for j in proyectos) <= h[i])

for j in proyectos:
    model.addConstr(quicksum(x[i, j] for i in trabajadores) >= l[j])
    model.addConstr(quicksum(x[i, j] for i in trabajadores) <= u[j])
    model.addConstr(quicksum(c[i, j] * x[i, j] for i in trabajadores) <= b[j])

# Objetivo
model.setObjective(
    quicksum((a[j] - 0.5 * c[i, j]) * x[i, j] for i in trabajadores for j in proyectos),
    GRB.MAXIMIZE
)

model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.5.0 24F74)

CPU model: Apple M4
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 28 rows, 60 columns and 240 nonzeros
Model fingerprint: 0x03ad7a4a
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [6e+01, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+00, 1e+03]
Presolve removed 12 rows and 0 columns
Presolve time: 0.00s
Presolved: 16 rows, 66 columns, 126 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    9.2320000e+03   5.660188e+02   0.000000e+00      0s
      22    9.2320000e+03   0.000000e+00   0.000000e+00      0s

Solved in 22 iterations and 0.00 seconds (0.00 work units)
Optimal objective  9.232000000e+03


In [8]:
# Resultados
for i in trabajadores:
    for j in proyectos:
        if x[i, j].X > 0:
            print(f"{i} trabaja {x[i, j].X:.1f} horas en {j}")
print(f"\nIngreso total: {model.ObjVal:.2f}")

Pedro trabaja 10.0 horas en Alto Trufen
María trabaja 7.0 horas en Alto Trufen
Luciana trabaja 14.7 horas en Villa Zeckers
Luciana trabaja 0.3 horas en Lo Redal
Claudia trabaja 14.0 horas en Tripamonti
Javier trabaja 15.0 horas en Rangekuta
Ignacia trabaja 9.0 horas en Omega
Lucas trabaja 9.9 horas en Lo Redal
Lucas trabaja 5.0 horas en Alto Trufen
Mariana trabaja 3.3 horas en Tripamonti
Mariana trabaja 1.3 horas en Omega
Mariana trabaja 5.3 horas en Lo Redal
Mateo trabaja 6.0 horas en Alto Trufen

Ingreso total: 9232.00
